<a href="https://colab.research.google.com/github/cbi-automation/lk-extraction/blob/fitriadc-dev/MVP_FinX_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 📄 FinX : Tools Ekstraksi Informasi Laporan Keuangan

Program ini dibuat untuk mengekstraksi informasi penting dari laporan keuangan (PDF) berbagai perusahaan yang terdaftar di Bursa Efek Indonesia. Dokumentasi ini ditujukan untuk pembaca non-teknis maupun teknis, agar dapat memahami alur logika program dan mengembangkannya lebih lanjut.

# Product Documentation

## System Documentation

### **Arsitektur Sistem**

FinX tersusun dari beberapa modul utama:

- `Main.ipynb`: Entry point untuk menjalankan proses ekstraksi.
- `marker_config.py`: Marker unik per emiten yang memandu proses ekstraksi.

### **Alur Data**

- **Input**: Laporan keuangan (PDF) dari situs BEI.
- **Proses**: Dibaca dengan `pdfplumber`, dipilah berdasarkan marker.
- **Output**: File `.xlsx` berisi informasi yang terstruktur.

## User Documentation

### **Instalasi Program**

- Tambahkan pintasan folder berikut ke Google Drive lokal (root):
   [📁 Folder Drive FinX](https://drive.google.com/drive/folders/1mZBcnBDgAkWPbhptWDl6UhLeUB6_ps6M)

- Buka link program berikut:
   [🔗 Program Utama FinX (Colab)](https://github.com/cbi-automation/lk-extraction/blob/fitriadc-dev/Main.ipynb)

- Jalankan langkah berikut:
   - Masukkan daftar nama emiten, kuarta, dan tahun yang ingin diesktrak pada variabel list `all_companies`, `kuartal`, dan `tahun`.
   - Klik **Runtime > Run all** dan tunggu proses selesai.

### **Menambah Emiten Baru**

- Buat folder baru pada folder `IDX_CALK_FX_SB_82`dengan format `/{kode_emiten}`
- Tambahkan file pdf di dalam folder emiten tersebut dengan format `{kode_emiten}-{tahun}-{kuartal}`
- Lakukan langkah `Menambah Jenis Informasi Baru untuk Diekstrak` sesuai jenis informasi yang dibutuhkan dari emiten tersebut.

### **Menambah Jenis Informasi Baru untuk Sebuah Emiten**

- Tentukan pattern marker (start dan end) yang tertera dalam dokument pdf sesuai jenis informasi yang ingin diekstrak.
- Tambahkan marker tersebut di `marker_config.py`.

# Source Code Documentation

#### Setup & Inisialisasi Lingkungan

Bagian pertama memuat berbagai *library* yang dibutuhkan, baik untuk pemrosesan file PDF (`pdfplumber`, `PyMuPDF`), pengolahan data (`pandas`), maupun manajemen sistem (`os`, `json`, `time`).

In [678]:
import os
import json
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
!pip install PyMuPDF
import warnings, logging, re, os
!pip install pdfplumber
import pdfplumber
import pandas as pd
from google.colab import files
import time
import fitz  # PyMuPDF
from dataclasses import dataclass, field
from typing import Dict, Optional

In [679]:
from google.colab import drive
drive.mount('/content/drive')

# Catat waktu mulai
start_program_time = time.time()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Pencatatan waktu program dijalankan dengan time.time(). Hal ini digunakan untuk mengetahui durasi total eksekusi program.

Selanjutnya, dilakukan konfigurasi path dasar sumber data dan folder output sebagai tempat menyimpan hasil ekstraksi. Jika belum tersedia, maka folder otomatis dibuat menggunakan `os.makedirs`.

In [680]:
# Path dasar input dan output
base_path = "/content/drive/MyDrive/IDX_CALK_FX_SB_82"
output_path = os.path.join(base_path, "output")
os.makedirs(output_path, exist_ok=True)

# Konfigurasi nama file output
output_json_file = "hasil_semua.json"
output_file = "hasil_ekstraksi.xlsx"
benchmark_file = "benchmark_waktu.xlsx"

#### Parameter Pemrosesan & Multi-threading

Daftar perusahaan yang akan diproses disimpan dalam `all_companies`, serta tahun (`years`) dan kuartal (`quarters`) yang diinginkan. Pemrosesan dilakukan secara paralel (multi-threading) dengan batas maksimal thread (`max_workers`) yang disesuaikan dengan jumlah inti CPU.

In [681]:
# Daftar perusahaan
# all_companies = [
#     "TLKM", "INDF", "INKP", "ADRO", "UNTR", "JSMR", "ICBP", "MEDC", "ISAT", "GIAA",
#     "PGAS", "EXCL", "TPIA", "SMGR", "MDKA", "BHIT", "BSDE", "WIKA", "IMAS", "TKIM",
#     "HMSP", "LPKR", "INDY", "BYAN", "INCO", "TBIG", "CTRA", "FREN", "EMTK", "CPIN",
#     "ANTM", "SMAR", "PTBA", "ADHI", "PANI", "HRUM", "AMRT", "BMTR", "SIMP", "JPFA",
#     "ITMG", "PWON", "ABMM", "AKRA", "AALI", "INTP", "KLBF", "MAPI", "BIPI", "APLN",
#     "MYOR", "DOID", "TBLA", "BUKA", "MNCN", "ENRG", "ASRI", "CMNP", "SMCB", "DNET",
#     "AUTO", "BKSL", "CENT", "GEMS", "SMDR", "GJTL", "UNVR", "BRMS", "CMNT", "KAEF",
#     "DSNG", "DUTI", "TOBA", "DILD", "TAPG", "MDLN", "NIRO", "JRPT", "PSAB", "KIJA",
#     "TINS", "LSIP", "ARCI", "MLPL", "FASW", "INDR", "PLIN", "MCOL", "TSPC", "SSMS",
#     "CSAP", "MAPA", "PTRO", "EPMT", "SILO", "SCMA", "LPCK", "MTDL", "ELSA", "SGRO"
# ]

all_companies = ["SMCB"]

# Tahun dan kuartal yang ingin diproses
years = ["2021", "2022","2023", "2024", "2025"]
quarters = ["Q1", "Q2", "Q3","Q4"]
max_workers = min(10, os.cpu_count() or 4)
batch_size = 10

import logging
logging.getLogger("pdfminer").setLevel(logging.ERROR)

#### Konfigurasi Marker untuk Ekstraksi Teks

Marker adalah kata kunci yang digunakan untuk menemukan bagian penting dari laporan keuangan. File `marker_config.py` diunduh langsung dari GitHub dan berisi konfigurasi penanda untuk berbagai perusahaan. Selanjutnya diambil konfigurasi marker untuk ratusan emiten saham dari file `marker_config.py`yang berisi aturan atau pola teks spesifik yang digunakan untuk mengekstraksi informasi tertentu dari laporan keuangan PDF emiten terkait. Setelah semua konfigurasi diimpor, kode tersebut membuat sebuah dictionary bernama marker_config_map yang memetakan ticker saham ke konfigurasi marker yang sesuai.

In [682]:
!wget https://raw.githubusercontent.com/cbi-automation/lk-extraction/fitriadc-dev/marker_config.py -O marker_config.py


from marker_config import (
    ELSA_marker_config,
    SGRO_marker_config,
    TLKM_marker_config,
    INDF_marker_config,
    INKP_marker_config,
    ADRO_marker_config,
    UNTR_marker_config,
    JSMR_marker_config,
    ICBP_marker_config,
    MEDC_marker_config,
    ISAT_marker_config,
    GIAA_marker_config,
    PGAS_marker_config,
    EXCL_marker_config,
    TPIA_marker_config,
    SMGR_marker_config,
    MDKA_marker_config,
    TOWR_marker_config,
    BHIT_marker_config,
    BSDE_marker_config,
    WIKA_marker_config,
    IMAS_marker_config,
    TKIM_marker_config,
    PTPP_marker_config,
    HMSP_marker_config,
    LPKR_marker_config,
    DSSA_marker_config,
    INDY_marker_config,
    BYAN_marker_config,
    INCO_marker_config,
    TBIG_marker_config,
    CTRA_marker_config,
    FREN_marker_config,
    EMTK_marker_config,
    KRAS_marker_config,
    CPIN_marker_config,
    ANTM_marker_config,
    SMAR_marker_config,
    PTBA_marker_config,
    ADHI_marker_config,
    PANI_marker_config,
    HRUM_marker_config,
    AMRT_marker_config,
    BMTR_marker_config,
    SIMP_marker_config,
    JPFA_marker_config,
    ITMG_marker_config,
    KPIG_marker_config,
    PWON_marker_config,
    ABMM_marker_config,
    SMRA_marker_config,
    AKRA_marker_config,
    AALI_marker_config,
    INTP_marker_config,
    KLBF_marker_config,
    MAPI_marker_config,
    BIPI_marker_config,
    APLN_marker_config,
    MYOR_marker_config,
    DOID_marker_config,
    TBLA_marker_config,
    BUKA_marker_config,
    MNCN_marker_config,
    ERAA_marker_config,
    ENRG_marker_config,
    ASRI_marker_config,
    CMNP_marker_config,
    SMCB_marker_config,
    POWR_marker_config,
    DNET_marker_config,
    AUTO_marker_config,
    BKSL_marker_config,
    CENT_marker_config,
    GEMS_marker_config,
    SMDR_marker_config,
    GJTL_marker_config,
    PPRO_marker_config,
    # GGRP_marker_config,
    UNVR_marker_config,
    BRMS_marker_config,
    CMNT_marker_config,
    KAEF_marker_config,
    DSNG_marker_config,
    DUTI_marker_config,
    TOBA_marker_config,
    DILD_marker_config,
    TAPG_marker_config,
    MDLN_marker_config,
    NIRO_marker_config,
    JRPT_marker_config,
    PSAB_marker_config,
    KIJA_marker_config,
    TINS_marker_config,
    LSIP_marker_config,
    ARCI_marker_config,
    MLPL_marker_config,
    FASW_marker_config,
    INDR_marker_config,
    PLIN_marker_config,
    MCOL_marker_config,
    TSPC_marker_config,
    SSMS_marker_config,
    CSAP_marker_config,
    MAPA_marker_config,
    PTRO_marker_config,
    EPMT_marker_config,
    SILO_marker_config,
    SCMA_marker_config,
    LPCK_marker_config,
    MTDL_marker_config
)

marker_config_map = {
    "ELSA":ELSA_marker_config,
    "SGRO": SGRO_marker_config,
    "TLKM": TLKM_marker_config,
    "INDF": INDF_marker_config,
    "INKP": INKP_marker_config,
    "ADRO": ADRO_marker_config,
    "UNTR": UNTR_marker_config,
    "JSMR": JSMR_marker_config,
    "ICBP": ICBP_marker_config,
    "MEDC": MEDC_marker_config,
    "ISAT": ISAT_marker_config,
    "GIAA": GIAA_marker_config,
    "PGAS": PGAS_marker_config,
    "EXCL": EXCL_marker_config,
    "TPIA": TPIA_marker_config,
    "SMGR": SMGR_marker_config,
    "MDKA": MDKA_marker_config,
    "TOWR": TOWR_marker_config,
    "BHIT": BHIT_marker_config,
    "BSDE": BSDE_marker_config,
    "WIKA": WIKA_marker_config,
    "IMAS": IMAS_marker_config,
    "TKIM": TKIM_marker_config,
    "PTPP": PTPP_marker_config,
    "HMSP": HMSP_marker_config,
    "LPKR": LPKR_marker_config,
    "DSSA": DSSA_marker_config,
    "INDY": INDY_marker_config,
    "BYAN": BYAN_marker_config,
    "INCO": INCO_marker_config,
    "TBIG": TBIG_marker_config,
    "CTRA": CTRA_marker_config,
    "FREN": FREN_marker_config,
    "EMTK": EMTK_marker_config,
    "KRAS": KRAS_marker_config,
    "CPIN": CPIN_marker_config,
    "ANTM": ANTM_marker_config,
    "SMAR": SMAR_marker_config,
    "PTBA": PTBA_marker_config,
    "ADHI": ADHI_marker_config,
    "PANI": PANI_marker_config,
    "HRUM": HRUM_marker_config,
    "AMRT": AMRT_marker_config,
    "BMTR": BMTR_marker_config,
    "SIMP": SIMP_marker_config,
    "JPFA": JPFA_marker_config,
    "ITMG": ITMG_marker_config,
    "KPIG": KPIG_marker_config,
    "PWON": PWON_marker_config,
    "ABMM": ABMM_marker_config,
    "SMRA": SMRA_marker_config,
    "AKRA": AKRA_marker_config,
    "AALI": AALI_marker_config,
    "INTP": INTP_marker_config,
    "KLBF": KLBF_marker_config,
    "MAPI": MAPI_marker_config,
    "BIPI": BIPI_marker_config,
    "APLN": APLN_marker_config,
    "MYOR": MYOR_marker_config,
    "DOID": DOID_marker_config,
    "TBLA": TBLA_marker_config,
    "BUKA": BUKA_marker_config,
    "MNCN": MNCN_marker_config,
    "ERAA": ERAA_marker_config,
    "ENRG": ENRG_marker_config,
    "ASRI": ASRI_marker_config,
    "CMNP": CMNP_marker_config,
    "SMCB": SMCB_marker_config,
    "POWR": POWR_marker_config,
    "DNET": DNET_marker_config,
    "AUTO": AUTO_marker_config,
    "BKSL": BKSL_marker_config,
    "CENT": CENT_marker_config,
    "GEMS": GEMS_marker_config,
    "SMDR": SMDR_marker_config,
    "GJTL": GJTL_marker_config,
    "PPRO": PPRO_marker_config,
    # "GGRP": GGRP_marker_config,
    "UNVR": UNVR_marker_config,
    "BRMS": BRMS_marker_config,
    "CMNT": CMNT_marker_config,
    "KAEF": KAEF_marker_config,
    "DSNG": DSNG_marker_config,
    "DUTI": DUTI_marker_config,
    "TOBA": TOBA_marker_config,
    "DILD": DILD_marker_config,
    "TAPG": TAPG_marker_config,
    "MDLN": MDLN_marker_config,
    "NIRO": NIRO_marker_config,
    "JRPT": JRPT_marker_config,
    "PSAB": PSAB_marker_config,
    "KIJA": KIJA_marker_config,
    "TINS": TINS_marker_config,
    "LSIP": LSIP_marker_config,
    "ARCI": ARCI_marker_config,
    "MLPL": MLPL_marker_config,
    "FASW": FASW_marker_config,
    "INDR": INDR_marker_config,
    "PLIN": PLIN_marker_config,
    "MCOL": MCOL_marker_config,
    "TSPC": TSPC_marker_config,
    "SSMS": SSMS_marker_config,
    "CSAP": CSAP_marker_config,
    "MAPA": MAPA_marker_config,
    "PTRO": PTRO_marker_config,
    "EPMT": EPMT_marker_config,
    "SILO": SILO_marker_config,
    "SCMA": SCMA_marker_config,
    "LPCK": LPCK_marker_config,
    "MTDL": MTDL_marker_config
}

--2025-06-12 04:13:57--  https://raw.githubusercontent.com/cbi-automation/lk-extraction/fitriadc-dev/marker_config.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23469 (23K) [text/plain]
Saving to: ‘marker_config.py’

marker_config.py    100%[===================>]  22.92K  --.-KB/s    in 0.008s  

2025-06-12 04:13:57 (2.79 MB/s) - ‘marker_config.py’ saved [23469/23469]



#### Ekstraksi Informasi & Parsing Dokumen

Fungsi `extract_text()` bertanggung jawab untuk membaca seluruh isi teks dari setiap halaman PDF. Library fitz (alias PyMuPDF) digunakan karena lebih cepat dan andal dibanding library lain seperti pdfminer.

In [683]:
# Fungsi ekstraksi teks yang cepat
def extract_text(file_path):
    text = ""
    with fitz.open(file_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

Program menggunakan Python `@dataclass` untuk menyimpan hasil ekstraksi dalam bentuk struktur data Company. Ini memudahkan penyimpanan dan konversi ke format tabular seperti CSV atau Excel. Setiap objek `Company` merepresentasikan satu laporan keuangan dari satu perusahaan pada satu kuartal.

In [684]:
from dataclasses import dataclass, asdict
from typing import Dict

@dataclass
class Company:
    kuartal: str = "-"
    perusahaan: str = "-"
    satuan_nilai_tukar: str = "-"
    perubahan_laba_rugi_naik: str = "-"
    perubahan_laba_rugi_turun: str = "-"
    normalisasi_laba_rugi_naik: str = "-"
    normalisasi_laba_rugi_turun: str = "-"
    normalisasi_laba_rugi_tahun_sebelumnya_naik: str = "-"
    normalisasi_laba_rugi_tahun_sebelumnya_turun: str = "-"
    risiko_nilai_tukar: str = "-"
    risiko_suku_bunga: str = "-"
    satuan: str = "-"
    elastisitas_naik: str = "-"  # Tambahan
    elastisitas_turun: str = "-"  # Tambahan

    def to_dict(self) -> Dict[str, str]:
        return asdict(self)

    @classmethod
    def from_dict(cls, data: dict) -> 'Company':
        return cls(**data)

field_map: Dict[str, str] = {
    "kuartal": "Kuartal",
    "perusahaan": "Perusahaan",
    "satuan_nilai_tukar": "Satuan Nilai Tukar (persen)",
    "perubahan_laba_rugi_naik": "Perubahan Laba Rugi Naik",
    "perubahan_laba_rugi_turun": "Perubahan Laba Rugi Turun",
    "normalisasi_laba_rugi_naik": "Normalisasi Laba Rugi Naik",
    "normalisasi_laba_rugi_turun": "Normalisasi Laba Rugi Turun",
    "normalisasi_laba_rugi_tahun_sebelumnya_naik": "Normalisasi Laba Rugi Naik Tahun Sebelumnya",
    "normalisasi_laba_rugi_tahun_sebelumnya_turun": "Normalisasi Laba Rugi Turun Tahun Sebelumnya",
    "elastisitas_naik": "Elastisitas Naik",  # Tambahan
    "elastisitas_turun": "Elastisitas Turun",  # Tambahan
}


def get_str(value) -> str:
    if value is None:
        return "-"
    val = str(value).strip()
    return val if val else "-"

def generate_company(doc: Optional[dict]) -> Company:
    data = {}
    for attr, source_key in field_map.items():
        data[attr] = get_str(doc.get(source_key)) if doc else "-"
    return Company(**data)

company = Company()

`field_map` adalah dictionary yang berfungsi untuk memetakan nama atribut internal (misalnya risiko_suku_bunga) ke nama kolom hasil ekstraksi yang lebih manusiawi (seperti "Efek Risiko Suku Bunga"). Hal ini membuat hasil akhir lebih mudah dibaca dan diekspor ke Excel atau antarmuka lain.

Fungsi `get_str()` memastikan bahwa nilai dari hasil ekstraksi selalu berupa string yang bersih dari spasi atau None. Jika nilai kosong atau tidak tersedia, fungsi ini akan mengembalikan tanda -, agar hasil tetap konsisten dan tidak menimbulkan error saat diproses lebih lanjut.

Fungsi `generate_company()` membentuk satu instance Company berdasarkan hasil ekstraksi teks PDF dan marker yang sesuai untuk masing-masing perusahaan. Ia memanggil fungsi-fungsi find_satuan, find_nilai_tukar, dan find_suku_bunga dari konfigurasi marker emiten.

In [685]:

def find_paragraphs_by_marker_pairs(text, marker_pairs, kuartal="2022"):
    start_marker, end_marker = marker_pairs[0]
    start_idx = text.find(start_marker)
    if start_idx == -1:
        print(f"[❗] Start marker tidak ditemukan di {kuartal}: {start_marker}")
        return "-"  # Jika tidak ditemukan, kembalikan "-" sebagai indikator tidak ditemukan

    search_range = text[start_idx:]
    end_relative = search_range.find(end_marker)
    if end_relative == -1:
        print(f"[❗] End marker tidak ditemukan setelah start marker di {kuartal}: {end_marker}")
        return "-"  # Jika tidak ditemukan, kembalikan "-" sebagai indikator tidak ditemukan

    end_idx = start_idx + end_relative

    orig_start_idx = text.lower().find(start_marker.lower())
    orig_end_idx = text.lower().find(end_marker.lower(), orig_start_idx)

    if orig_start_idx != -1 and orig_end_idx != -1:
        content_raw = text[orig_start_idx + len(start_marker): orig_end_idx]
        snippet = content_raw.strip()
    else:
        snippet = text[start_idx + len(start_marker): end_idx].strip()

    return snippet  # Kembalikan string langsung, bukan list atau dict

Fungsi `process_pdf_batch()` menangani pemrosesan banyak file PDF secara paralel dalam batch. Ia akan mencoba mengimpor konfigurasi marker perusahaan secara dinamis, mengeksekusi generate_company, dan menyimpan hasil ekstraksi ke list `results`.

In [686]:
def find_suku_bunga(text, marker_pairs, company: Company, kuartal: str):
    # Gabungkan newline jadi spasi agar regex lebih fleksibel
    teks_kotor = find_paragraphs_by_marker_pairs(text, marker_pairs, kuartal)

    # Cek apakah teks_kotor adalah list, dan gabungkan menjadi string jika iya
    if isinstance(teks_kotor, list):
        teks_kotor = " ".join(teks_kotor)

    teks_bersih = teks_kotor.replace("\n", " ")

    company.risiko_suku_bunga = teks_bersih

Fungsi `find_nilai_tukar` dan `find_suku_bunga` bertanggung jawab untuk mengekstrak teks tertentu berdasarkan marker yang diberikan, yaitu terkait dengan risiko nilai tukar dan suku bunga. Keduanya menggunakan `find_paragraphs_by_marker_pairs` untuk mencari paragraf yang relevan dalam teks berdasarkan pasangan marker yang diberikan.

In [687]:
def process_all_markers(text, kuartal, emiten):
    company = Company()
    company.perusahaan = emiten
    company.kuartal = kuartal

    markers = marker_config_map.get(emiten.upper())

    for marker_name, marker_pairs in markers.items():
        function_name = marker_to_function.get(marker_name)
        if not function_name:
            continue

        func = globals().get(function_name)
        if not func:
            print(f"[⚠️] Fungsi '{function_name}' tidak ditemukan.")
            continue

        # ⬇️ Fungsi sekarang juga menerima objek company yang akan di-update
        func(text, marker_pairs, company, kuartal)

    return company

# Mapping marker ke fungsi
marker_to_function = {
    "marker1": "find_satuan",
    "marker2": "find_nilai_tukar",
    "marker3": "find_suku_bunga"
}

def find_satuan(text, marker_pairs, company: Company, kuartal: str):
    results = find_paragraphs_by_marker_pairs(text, marker_pairs,kuartal)
    # satuan = results if results else "-"
    company.satuan = results

Fungsi `process_all_markers` menerima teks laporan, kuartal, dan nama emiten, lalu mengiterasi markers yang relevan untuk emiten tersebut. Setiap marker dipetakan ke fungsi yang sesuai menggunakan `marker_to_function`, dan kemudian fungsi itu dipanggil untuk mengekstrak data yang relevan. Fungsi-fungsi yang dipanggil akan memperbarui objek `Company` dengan hasil ekstraksi.

In [688]:
# Fungsi pemrosesan individual
def process_file(emiten, year, quarter):
    kuartal_label = f"{year}-{quarter}"
    pdf_file = f"{base_path}/{emiten}/{emiten}-{year}-{quarter}.pdf"
    if not os.path.exists(pdf_file):
        print(f"[⚠️] File tidak ditemukan: {pdf_file}")
        return None

    print(f"\n🔍 Memproses: {emiten} - {kuartal_label}")
    try:
        start_time = time.time()
        text = extract_text(pdf_file)
        hasil = process_all_markers(text, kuartal_label, emiten)
        elapsed = time.time() - start_time
        print(f"\n✅ {kuartal_label}-{emiten} berhasil diproses dalam {elapsed:.2f} detik.")
        return f"{emiten}-{kuartal_label}", hasil.__dict__
    except Exception as e:
        print(f"[⚠️] Error pada {emiten}-{kuartal_label}: {e}")
        return None

Fungsi `process_file` menerima informasi emiten, tahun, dan kuartal, serta nama file PDF yang relevan. Jika file ditemukan, teks diekstraksi menggunakan `extract_text`, kemudian diproses dengan memanggil process_all_markers untuk mengekstrak data berdasarkan marker. Durasi pemrosesan dihitung dan dicatat

Dilakukan pemeriksaan terlebih dahulu, apakah file JSON hasil proses sebelumnya `(output_json_file`) ada. Jika file ada, maka isinya akan dimuat ke dalam variabel all_results_dicts_dinamis. Jika file tidak ada, variabel ini diinisialisasi sebagai dictionary kosong. Ini memastikan bahwa data yang sudah diproses sebelumnya tidak diproses ulang.

In [689]:
import os
import json

# Cek dan baca file JSON jika sudah ada
if os.path.exists(output_json_file):
    with open(output_json_file, "r", encoding="utf-8") as f:
        raw_data = json.load(f)
        all_results_dicts_dinamis = {
            k: Company.from_dict(v) for k, v in raw_data.items()
        }
else:
    all_results_dicts_dinamis = {}

# Simpan waktu proses per kuartal
benchmark_data = []

# Cek apakah kombinasi emiten-year-quarter sudah diproses
def is_processed(emiten, year, quarter):
    key = f"{emiten}-{year}-{quarter}"
    return key in all_results_dicts_dinamis

Fungsi `process_batch` memproses satu batch emiten dalam satu kuartal. Untuk setiap emiten dalam batch, fungsi ini memeriksa apakah laporan sudah diproses. Jika belum, fungsi ini memanggil `process_file` untuk memproses file laporan keuangan emiten tersebut. Setelah pemrosesan selesai, waktu yang digunakan untuk memproses laporan dihitung dan hasilnya disimpan dalam `batch_results`.

In [690]:
def process_batch(batch_emiten, year, quarter):
    batch_results = []
    for emiten in batch_emiten:
      if not is_processed(emiten, year, quarter):
            start_time = time.time()
            result = process_file(emiten, year, quarter)
            if result:
                key, data = result
                batch_results.append((key, data, time.time() - start_time))
    return batch_results

In [691]:
if os.path.exists(output_json_file):
    with open(output_json_file, "r", encoding="utf-8") as f:
        all_results_dicts_dinamis = json.load(f)
else:
    all_results_dicts_dinamis = {}

# Simpan waktu proses per kuartal
benchmark_data = []

# Cek apakah sudah diproses
def is_processed(emiten, year, quarter):
    key = f"{emiten}-{year}-{quarter}"
    return key in all_results_dicts_dinamis

Daftar emiten dibagi menjadi beberapa batch berdasarkan ukuran batch '(batch_size)'. Ini membantu untuk mengelompokkan emiten dalam jumlah yang lebih kecil sehingga proses dapat dijalankan secara paralel dan lebih efisien.

In [692]:
# Bagi jadi batch
emiten_batches = [all_companies[i:i + batch_size] for i in range(0, len(all_companies), batch_size)]

In [693]:
def isi_data_tahun_sebelumnya(perusahaan, kuartal, tahun):
    tahun_lalu = str(int(tahun) - 1)
    key_tahun_lalu = f"{perusahaan}-{tahun_lalu}-{kuartal}"
    data_obj = all_results_dicts_dinamis.get(key_tahun_lalu)

    try:
        company.normalisasi_laba_rugi_tahun_sebelumnya_naik = data_obj.normalisasi_laba_rugi_naik
        company.normalisasi_laba_rugi_tahun_sebelumnya_turun = data_obj.normalisasi_laba_rugi_turun
    except (AttributeError, TypeError):
        company.normalisasi_laba_rugi_tahun_sebelumnya_naik = "-"
        company.normalisasi_laba_rugi_tahun_sebelumnya_turun = "-"
        print(f"Data tahun sebelumnya tidak ditemukan atau tidak lengkap untuk {key_tahun_lalu}")

Untuk setiap batch emiten, kode menggunakan `ThreadPoolExecutor` untuk menjalankan proses pemrosesan laporan secara paralel. Setiap kuartal dan tahun diproses secara bersamaan untuk setiap emiten dalam batch. `max_workers ` menentukan jumlah thread yang akan digunakan secara bersamaan untuk pemrosesan ini.

### Compiling Nilai Tukar

In [694]:
import re
import requests

API_KEY = '0Ycbxo5RqcH71TAZJt6MBkG17sM8lX6U'

NILAI_TUKAR_HISTORY = {
    '2021-Q1': 14_200,
    '2021-Q2': 14_300,
    '2021-Q3': 14_500,
    '2021-Q4': 14_600,
    '2022-Q1': 14_700,
    '2022-Q2': 14_800,
    '2022-Q3': 15_000,
    '2022-Q4': 15_200,
    '2023-Q1': 14_968.65,
    '2023-Q2': 15_040.30,
    '2023-Q3': 15_493.00,
    '2023-Q4': 15_482.97,
    '2024-Q1': 15_205.00,
    '2024-Q2': 16_359.04,
    '2024-Q3': 15_321.63,
    '2024-Q4': 16_052.34,
    '2025-Q1': 16_466.11,
    '2025-Q2': 16_650,
}

KATEGORI_SATUAN_PERUSAHAAN = {
    "AMRT": "Jutaan Rupiah",
    "ADRO": "Ribuan USD",
    "MEDC": "USD",
    "GIAA": "USD",
    "PGAS": "USD",
    "MDKA": "USD",
    "BYAN": "USD",
    "INDY": "USD",
    "TLKM": "miliaran Rupiah",
    "EXCL": "jutaan Rupiah",
    "INDF": "Jutaan Rupiah",
    "ISAT": "Jutaan Rupiah",
    "SMGR": "jutaan Rupiah",
    "HMSP": "jutaan Rupiah",
    "LPKR": "Jutaan Rupiah",
    "BHIT": "jutaan Rupiah",
    "BSDE": "Rupiah",
    "IMAS": "Jutaan Rupiah",
    "TBIG": "jutaan Rupiah",
    "FREN": "Jutaan Rupiah",
    "INCO": "Ribuan USD",
    "TKIM":"Ribuan USD",
    "EMTK": "Ribuan rupiah",
    "SMAR": "Jutaan rupiah",
    "WIKA": "Ribuan rupiah",
    "PWON": "rupiah",
    "ITMG": "ribuan USD",
    "BMTR":"Jutaan rupiah",
    "JPFA": "jutaan rupiah",
    "ASRI" : "ribuan rupiah",
    "AALI" : "jutaan rupiah",
    "INTP" : "jutaan rupiah",
    "KLBF" : "rupiah",
    "BIPI" :"rupiah",
    "ASRI" : "ribuan rupiah",
    "CMNP" : "ribuan rupiah",
    "MLPL" : "ribuan rupiah",
    "LSIP":  "jutaan rupiah",
    "SMCB" : "jutaan rupiah"
}

In [695]:
def jutaan_to_satuan(nilai):
    return nilai * 1_000_000


def miliar_to_satuan(nilai):
    return nilai * 1_000_000_000


def ribuan_to_satuan(nilai):
    return nilai * 1_000


def usd_to_rupiah(nominal_usd: float, kuartal: str = None):
    """
    Mengonversi nominal USD ke IDR.
    Jika kuartal disediakan dan ada di history, gunakan nilai tukar historis.
    Jika tidak, fallback ke API real-time.
    """
    if kuartal and kuartal in NILAI_TUKAR_HISTORY:
        rate = NILAI_TUKAR_HISTORY[kuartal]
        amount_idr = nominal_usd * rate
        print(f"[HISTORIS] Nilai tukar untuk {kuartal}: 1 USD = {rate:,.0f} IDR")
        print(f"{nominal_usd:,.0f} USD = {amount_idr:,.0f} IDR (historis)")
        return (f"${amount_idr}")
    else:
        # Fallback: real-time rate via API
        url = 'https://api.apilayer.com/exchangerates_data/convert'
        headers = {
            'apikey': API_KEY
        }
        params = {
            'from': 'USD',
            'to': 'IDR',
            'amount': nominal_usd
        }
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            amount_idr = data.get('result')
            rate = data.get('info', {}).get('rate', '-')
            print(f"[REAL-TIME] Nilai tukar saat ini: 1 USD = {rate:,.2f} IDR")
            return amount_idr
        else:
            print("Gagal mendapatkan data nilai tukar:", response.text)
            return None


def standarisasi_nilai(perusahaan, nilai):
    satuan = KATEGORI_SATUAN_PERUSAHAAN.get(perusahaan)
    # Normalisasi input nilai
    nilai = str(nilai).lower().replace("as$", "").replace("$", "").replace(".", "").replace(",", "").strip()

    if satuan is None:
      satuan = "rupiah"
    else :
      satuan = satuan.lower()

    # Normalisasi ke satuan biasa
    if satuan == "jutaan rupiah":
        nilai_final = f"Rp{jutaan_to_satuan(float(nilai)):,.0f}"
    elif satuan == "miliaran rupiah":
        nilai_final = f"Rp{miliar_to_satuan(float(nilai)):,.0f}"
    elif satuan == "rupiah":
        nilai_final = f"Rp{float((nilai)):,.0f}"
    elif satuan == "ribuan usd":
        nilai_final = f"${ribuan_to_satuan(float(nilai)):,.0f}"
    elif satuan == "ribuan rupiah":
        nilai_final = f"Rp{ribuan_to_satuan(float(nilai)):,.0f}"
    elif satuan == "usd":
        nilai_final = f"${float(nilai):,.0f}"
    else:
        raise ValueError(f"Satuan {satuan} tidak dikenali.")
    return nilai_final

def konversi_mata_uang(nilai, perusahaan, kuartal: str):
    satuan = KATEGORI_SATUAN_PERUSAHAAN.get(perusahaan)
    nilai_str = str(nilai)
    nilai_clean = nilai_str.replace("$", "").replace(".", "").strip().replace(",", "").strip()
    try:
      nominal_usd = float(nilai_clean)
    except ValueError:
      raise ValueError(f"Nilai USD tidak valid: {nilai}")
    nilai_final = usd_to_rupiah(nominal_usd, kuartal)

    return nilai_final

In [696]:
def find_nilai_tukar(text, marker_pairs, company: Company, kuartal: str):
    perusahaan = company.perusahaan
    teks_kotor = find_paragraphs_by_marker_pairs(text, marker_pairs, kuartal)
    teks_bersih = teks_kotor.replace("\n", " ")
    company.risiko_nilai_tukar = teks_bersih
    print("INI KUARTAL:"+ kuartal)
    persen = "-"
    rupiah = "Rp0"

    print("INI BERHASIL 1")

    if company.perusahaan in ["WIKA","EMTK","INDF", "TBIG","FREN","JPFA","KLBF","MLPL","LSIP"]:
        persen_match = re.search(r"(\d+(?:[.,]\d+)?%)", teks_bersih, re.IGNORECASE)
        persen = persen_match.group(1) if persen_match else "-"
        rupiah_match = re.search(r"Rp\s?([\d\.,]+)", teks_bersih, re.IGNORECASE)
        if rupiah_match:
            # Konversi ke float dulu, baru pakai fungsi konversi ke rupiah
            nilai = float(rupiah_match.group(1).replace(".", "").replace(",", "."))
            rupiah = standarisasi_nilai(company.perusahaan, nilai)
        else:
            rupiah = "Rp0"
        company.perubahan_laba_rugi_turun = rupiah
        company.normalisasi_laba_rugi_turun = rupiah
        company.normalisasi_laba_rugi_naik = rupiah
        print("INI BERHASIL 2")

    elif company.perusahaan in ["BMTR"]:
        persen_match = re.search(r"(\d+(?:[.,]\d+)?%)", teks_bersih, re.IGNORECASE)
        persen = persen_match.group(1) if persen_match else "-"
        rupiah_match = re.search(r"(penguatan|pelemahan)\s*(\d+%)\s*\(?([\d.]+)\)?", teks_bersih, re.IGNORECASE)
        if rupiah_match:
            nilai_str = rupiah_match.group(3)
            nilai = float(nilai_str.replace(".", "").replace(",", "."))
            rupiah = standarisasi_nilai(perusahaan, nilai)
        else:
            rupiah = "Rp0"
        company.perubahan_laba_rugi_turun = rupiah
        company.normalisasi_laba_rugi_turun = rupiah
        company.normalisasi_laba_rugi_naik = rupiah
        print("INI BERHASIL 3")
    elif company.perusahaan == "TLKM":
        # Ambil angka nominal (misal 92) di konteks Dolar A.S.
        persen_match = re.search(r"Dolar A\.S\.\s+\(penguatan\s+(\d+%)\)\s+\(?(-?\d+[\.,]?\d*)\)?", teks_bersih, re.IGNORECASE)
        persen = persen_match.group(1) if persen_match else "-"
        nominal_str = persen_match.group(2).replace(",", ".") if persen_match else "0"
        try:
            nominal_float = float(nominal_str)
        except ValueError:
            nominal_float = 0

        rupiah = standarisasi_nilai(company.perusahaan, nominal_float)
        company.perubahan_laba_rugi_turun = rupiah
        company.normalisasi_laba_rugi_turun = rupiah
        company.normalisasi_laba_rugi_naik = rupiah


    elif perusahaan in ["MEDC", "TKIM", "INDY", "BYAN"]:
        persen_match = re.search(r"(\d+(?:[.,]\d+)?%)", teks_bersih)
        persen = persen_match.group(1) if persen_match else "-"
        rupiah_match = re.search(r"(?:AS\$|US\$|USD|\$)\s*([\d.,]+)", teks_bersih)
        usd_nominal = rupiah_match.group(1) if rupiah_match else "$0"
        rupiah = f"${usd_nominal}"

        nominal_rupiah = konversi_mata_uang(usd_nominal, perusahaan, kuartal)
        company.perubahan_laba_rugi_turun = rupiah
        company.normalisasi_laba_rugi_turun = nominal_rupiah
        company.normalisasi_laba_rugi_naik = nominal_rupiah


    elif company.perusahaan in ["INCO"]:
        persen_match = re.search(r"sebesar\s+(\d+(?:[.,]\d+)?)%", teks_bersih, re.IGNORECASE)
        persen = f"{persen_match.group(1)}%" if persen_match else "-"

        rupiah_match = re.search(r"(AS\$[\d\.,]+\s*juta)", teks_bersih, re.IGNORECASE)
        usd_nominal = rupiah_match.group(1) if rupiah_match else "0"

        nominal_usd = standarisasi_nilai(company.perusahaan, usd_nominal)
        rupiah = nominal_usd

        nominal_rupiah = konversi_mata_uang(nominal_usd, perusahaan, kuartal)
        company.perubahan_laba_rugi_turun = rupiah
        company.normalisasi_laba_rugi_turun = nominal_rupiah
        company.normalisasi_laba_rugi_naik = nominal_rupiah

    elif company.perusahaan in ["IMAS", "LPKR"]:
        persen_match = re.search(r"(?:sebanyak|by|sebesar)\s+(\d+(?:[.,]\d+)?%)", teks_bersih, re.IGNORECASE)
        persen = persen_match.group(1) if persen_match else "-"
        rupiah_match = re.search(r"Rp[\s\.]*([\d\.,]+)", teks_bersih, re.IGNORECASE)
        rupiah = (
          standarisasi_nilai(company.perusahaan, float(rupiah_match.group(1).replace(".", "").replace(",", ".")))
          if rupiah_match
          else "Rp0"
        )
        company.perubahan_laba_rugi_turun = rupiah
        company.normalisasi_laba_rugi_turun = rupiah
        company.normalisasi_laba_rugi_naik = rupiah

    elif perusahaan == "HMSP":
        persen_match = re.search(r"(Rp\d+(?:[.,]\d+)?\/1USD)", teks_bersih)
        persen = persen_match.group(1) if persen_match else "-"
        rupiah_match = re.search(r"Rp\s*([\d\.,]+)\s*miliar", teks_bersih, re.IGNORECASE)
        if rupiah_match:
            angka_str = rupiah_match.group(1).replace(".", "").replace(",", ".")
            try:
                nominal = float(angka_str) * 1_000_000_000  # miliar ke satuan
                rupiah = standarisasi_nilai(perusahaan, nominal)
            except ValueError:
                rupiah = "Rp0"
        company.perubahan_laba_rugi_turun = rupiah
        company.normalisasi_laba_rugi_turun = rupiah
        company.normalisasi_laba_rugi_naik = rupiah

    elif perusahaan in ["GIAA","INTP"]:
        rupiah_match = re.search(r"Rupiah\s+\d+%\s+\(?([\d,\.]+)\)?", teks_bersih)
        persen_match = re.search(r"(\d+)%", teks_bersih)

        if rupiah_match:
            persen = persen_match.group(1) + "%" if persen_match else "-"
            angka_str = rupiah_match.group(1).replace(",", "").replace(".", "")
            try:
                nominal = float(angka_str)
                nominal_idr = konversi_mata_uang(nominal, perusahaan, kuartal)
                rupiah = standarisasi_nilai(perusahaan, nominal)
                company.normalisasi_laba_rugi_turun = nominal_idr
                company.normalisasi_laba_rugi_naik = nominal_idr
            except ValueError:
                rupiah = "-"
        else:
            rupiah = "-"

        company.perubahan_laba_rugi_turun = rupiah


    elif company.perusahaan == "SMGR":
        persen_match = re.search(r"(\d+(?:[.,]\d+)?%)", teks_bersih, re.IGNORECASE)
        persen = persen_match.group(1) if persen_match else "-"
        rupiah_match = re.search(r"(?:Rp)?\s?([\d\.,]+)", teks_bersih, re.DOTALL)
        rupiah = rupiah_match.group(1) if rupiah_match else "Rp0"
        company.perubahan_laba_rugi_turun = rupiah
        company.normalisasi_laba_rugi_turun = rupiah
        company.normalisasi_laba_rugi_naik = rupiah

    elif company.perusahaan in ["MDKA", "AALI"]:
        persen_match = re.search(r"\d+(?:[.,]\d+)?%", teks_bersih)
        persen = persen_match.group(0) if persen_match else "-"

        angka = re.findall(r"\d{1,3}(?:[.,]\d{3})+", teks_bersih)
        if angka:
            nominal_usd = float(angka[0].replace(".", "").replace(",", ""))
            nominal_idr = konversi_mata_uang(nominal_usd, perusahaan, kuartal)
            rupiah = standarisasi_nilai(perusahaan, nominal_usd)
            company.normalisasi_laba_rugi_turun = nominal_idr
            company.normalisasi_laba_rugi_naik = nominal_idr
        else:
            rupiah = "-"
        company.perubahan_laba_rugi_turun = rupiah


    elif company.perusahaan == "BHIT":
        persen_match = re.search(r"Penguatan\s+(\d+,\d+)%", teks_bersih, re.IGNORECASE)
        persen = persen_match.group(1) + "%" if persen_match else "-"

        rupiah_match = re.search(r"Penguatan\s+\d+,\d+%\s+([\d\.]+)", teks_bersih, re.IGNORECASE)
        rupiah = (
            standarisasi_nilai(company.perusahaan, float(rupiah_match.group(1).replace(".", "")))
            if rupiah_match
            else "-"
        )

        rupiah_turun_match = re.search(r"Pelemahan\s+\d+,\d+%\s+\(([\d\.]+)\)", teks_bersih, re.IGNORECASE)
        rupiah_turun = (
            "-" + standarisasi_nilai(company.perusahaan, float(rupiah_turun_match.group(1).replace(".", "")))
            if rupiah_turun_match
            else "-"
        )
        company.perubahan_laba_rugi_turun = rupiah_turun
        company.normalisasi_laba_rugi_turun = rupiah_turun
        company.normalisasi_laba_rugi_naik = rupiah

    elif company.perusahaan == "BSDE":

        matches = re.findall(r"Rp\s?([\d.,]{7,})", teks_bersih)
        rupiah = f"Rp{matches[0]}" if matches else "Rp0"
        company.perubahan_laba_rugi_turun = rupiah
        company.normalisasi_laba_rugi_turun = rupiah
        company.normalisasi_laba_rugi_naik = rupiah

    elif company.perusahaan == "ISAT":
        persen_match = re.search(r"(?:Dollar)\s+(\d+(?:\.\d+)?%)", teks_bersih, re.IGNORECASE)
        persen = persen_match.group(1) if persen_match else "-"
        laba_rugi_match = re.search(r"(?:berjalan)\s*(\d+(?:[.,]\d+)?)", teks_bersih, re.IGNORECASE)
        if laba_rugi_match:
            nominal_ribuan = float(laba_rugi_match.group(1).replace(",", "."))
            # Konversi ke USD (dalam ribuan) lalu ke IDR real-time
            nominal_usd = ribuan_to_satuan(nominal_ribuan)
            rupiah =  f"Rp{nominal_usd}"
            nominal_idr = konversi_mata_uang(nominal_usd,perusahaan, kuartal)
            company.normalisasi_laba_rugi_turun = nominal_idr
            company.normalisasi_laba_rugi_naik = nominal_idr
        else:
            rupiah = "-"
        company.perubahan_laba_rugi_turun = rupiah


    elif company.perusahaan == "WIKA":
        print("INI TEKS BERSIH " + teks_bersih)
        persen_match = re.search(r"(\d+%)", teks_bersih)
        persen = persen_match.group(1) if persen_match else "-"
        rupiah_match = re.search(r"Rp[\d\.,]+", teks_bersih)
        if rupiah_match:
            rupiah_idr = rupiah_match.group(0).replace('Rp', '').strip().replace(',', '.')
            print("NILAI ASLI " + rupiah_idr)
            rupiah = standarisasi_nilai(perusahaan, rupiah_idr)
            print("STANDARISASI NILAI "+rupiah)
            company.normalisasi_laba_rugi_turun = rupiah
            print("NORMALISASI LABA RUGI TURUN" + company.normalisasi_laba_rugi_tahun_sebelumnya_turun)
            company.normalisasi_laba_rugi_naik = rupiah
            print("NORMALISASI LABA RUGI NAIK" + company.normalisasi_laba_rugi_tahun_sebelumnya_naik)
        else:
            rupiah = "-"
        company.perubahan_laba_rugi_turun = rupiah


    elif company.perusahaan in ["ADRO","ITMG"]:
        # Ambil persen
        persen_match = re.search(r"(\d+(?:\.\d+)?%)", teks_bersih)
        persen = persen_match.group(1) if persen_match else "-"

        # Ambil nilai 'lebih rendah' dalam ribuan USD
        rupiah_turun_match = re.search(r"lebih rendah\s*AS\$([\d\.,]+)", teks_bersih, re.IGNORECASE)
        if rupiah_turun_match:
            nominal_ribuan = float(rupiah_turun_match.group(1).replace(",", "").replace(".", "."))
            nominal_usd = standarisasi_nilai(perusahaan, nominal_ribuan)
            rupiah_turun = nominal_usd
            nominal_idr = konversi_mata_uang(nominal_usd,perusahaan,  kuartal)  # konversi ke IDR
            company.normalisasi_laba_rugi_turun = nominal_idr
        else:
            rupiah_turun = "-"

        # Ambil nilai 'lebih tinggi' dalam ribuan USD
        rupiah_naik_match = re.search(r"lebih tinggi\s*AS\$([\d\.,]+)", teks_bersih, re.IGNORECASE)
        if rupiah_naik_match:
            nominal_ribuan = float(rupiah_naik_match.group(1).replace(",", "").replace(".", "."))
            nominal_usd = standarisasi_nilai(perusahaan, nominal_ribuan)
            rupiah = nominal_usd
            nominal_idr = konversi_mata_uang(nominal_usd,perusahaan,kuartal)
            company.normalisasi_laba_rugi_naik = nominal_idr
        else:
            rupiah = "-"

        company.perubahan_laba_rugi_turun = rupiah_turun


    elif company.perusahaan == "PGAS":
        persen_match = re.search(r"(\d+\s*basis\s*point)", teks_bersih, re.IGNORECASE)
        persen = persen_match.group(1) if persen_match else "-"
        print("INI PERSEN: "+persen)

        # rupiah naik
        rupiah_naik_match = re.search(r"by\s*USD\s*([\d.,]+)", teks_bersih, re.IGNORECASE)
        if rupiah_naik_match:
            nominal_usd = float(rupiah_naik_match.group(1).replace(",", ""))
            print("INI NOMINAL USD: "+str(nominal_usd))
            nominal_usd_standarisasi = standarisasi_nilai(perusahaan, nominal_usd)
            print("INI NOMINAL USD STANDARISASI: "+nominal_usd_standarisasi)
            rupiah = nominal_usd_standarisasi
            nominal_idr = konversi_mata_uang(nominal_usd_standarisasi,perusahaan,  kuartal)  # konversi ke IDR
            company.normalisasi_laba_rugi_naik = nominal_idr
        else:
            rupiah = "-"


        # rupiah turun
        rupiah_turun_match = re.search(r"\(USD\s*([\d.,]+)\)", teks_bersih, re.IGNORECASE)
        if rupiah_turun_match:
            nominal_usd = float(rupiah_turun_match.group(1).replace(",", ""))
            print("INI NOMINAL USD TURUN: "+str(nominal_usd))
            nominal_usd_standarisasi = standarisasi_nilai(perusahaan, nominal_usd)
            print("INI NOMINAL USD STANDARISASI TURUN: "+nominal_usd_standarisasi)
            rupiah_turun = nominal_usd_standarisasi
            nominal_idr = konversi_mata_uang(nominal_usd_standarisasi,perusahaan,  kuartal)    # konversi ke IDR
            company.normalisasi_laba_rugi_turun = nominal_idr
        else:
            rupiah_turun = "-"

        company.perubahan_laba_rugi_turun = rupiah_turun

    elif company.perusahaan in ["EXCL", "SMAR"]:
        persen_match = re.search(r"(\d+(?:\.\d+)?%)", teks_bersih)
        persen = persen_match.group(1) if persen_match else "-"
        rupiah_match = re.search(r"Rp\s?([\d\.,]+)\s+dan\s+Rp\s?([\d\.,]+)", teks_bersih, re.IGNORECASE)
        if rupiah_match:
            # Ubah string ke float (ganti koma dengan titik)
            nilai_turun = float(rupiah_match.group(1).replace(".", "").replace(",", "."))
            nilai_naik = float(rupiah_match.group(2).replace(".", "").replace(",", "."))
            rupiah_turun = standarisasi_nilai(company.perusahaan,  nilai_turun)
            rupiah = standarisasi_nilai(company.perusahaan, nilai_naik)

        else:
            rupiah_turun = "Rp 0"
            rupiah = "Rp 0"
        company.perubahan_laba_rugi_turun = rupiah_turun
        company.normalisasi_laba_rugi_turun = rupiah_turun
        company.normalisasi_laba_rugi_naik = rupiah

    elif company.perusahaan == "PWON" :
        persen_match = re.search(r"(\d+(?:\.\d+)?%)", teks_bersih)
        persen = persen_match.group(1) if persen_match else "-"
        rupiah_match = re.search(r"Laba\s*rugi\s+([\d,.]+)", teks_bersih, re.IGNORECASE)
        if rupiah_match:
            nilai_turun = float(rupiah_match.group(1).replace(".", "").replace(",", "."))
            print("INI NILAI TURUN: "+str(nilai_turun))
            nilai_naik = float(rupiah_match.group(2).replace(".", "").replace(",", "."))
            print("INI NILAI NAIK: "+str(nilai_naik))
            rupiah_turun = standarisasi_nilai(company.perusahaan,  nilai_turun)
            print("INI RUPIAH TURUN: "+rupiah_turun)
            rupiah = standarisasi_nilai(company.perusahaan, nilai_naik)

        else:
            rupiah = "-"
        company.perubahan_laba_rugi_turun = rupiah_turun
        company.normalisasi_laba_rugi_turun = rupiah_turun
        company.normalisasi_laba_rugi_naik = rupiah

    elif perusahaan in ["AMRT","INTP"]:
        print("INI BERHASIL 5")
        persen_match = re.search(r"Rupiah\s+[+-]?(\d+)\s+[(-]?\d+", teks_bersih)
        persen = persen_match.group(1) if persen_match else "-"

        # Tangkap nilai naik dan turun
        naik_match = re.search(r"Rupiah\s*\+\d+\s*\(?(-?\d+)\)?", teks_bersih)
        turun_match = re.search(r"Rupiah\s*-\d+\s*\(?(-?\d+)\)?", teks_bersih)

        nilai_naik = int(naik_match.group(1).replace(",", "").replace(".", "")) if naik_match else 0
        nilai_turun = int(turun_match.group(1).replace(",", "").replace(".", "")) if turun_match else 0

        # Tambahkan tanda negatif jika ada tanda kurung
        if naik_match and "(" in naik_match.group(0):
            nilai_naik *= -1
        if turun_match and "(" in turun_match.group(0):
            nilai_turun *= -1

        # Konversi nilai ke format standarisasi jika perlu
        rupiah_naik = standarisasi_nilai(perusahaan, nilai_naik)
        rupiah_turun = standarisasi_nilai(perusahaan, nilai_turun)

        company.perubahan_laba_rugi_turun = rupiah_turun
        company.normalisasi_laba_rugi_turun = rupiah_turun
        company.normalisasi_laba_rugi_naik = rupiah_naik

    elif perusahaan in ["JPFA","KLBF","BIPI","ASRI","CMNP","SMCB"] :
      persen_match = re.search(r"(Rupiah\s+[+-]?\d+\s+[(-]?\d+)|(\d+(?:\.\d+)?%)", teks_bersih)
      persen = persen_match.group(1) if persen_match else "-"

      if perusahaan == "ASRI" :

          # Ambil nilai 'lebih rendah sebesar Rp ...'
        turun_match = re.search(r"lebih rendah.*?Rp\s*([\d.]+)", teks_bersih, re.IGNORECASE)
        nilai_turun = int(turun_match.group(1).replace(".", "")) if turun_match else 0

        # Ambil nilai 'lebih tinggi.*?sebesar Rp ...'
        naik_match = re.search(r"lebih tinggi.*?sebesar Rp\s*([\d.]+)", teks_bersih, re.IGNORECASE)
        nilai_naik = int(naik_match.group(1).replace(".", "")) if naik_match else 0

      elif perusahaan in ["AMRT","BIPI"] :
          naik_match = re.search(r"Rp meningkat sebesar\s+\d+%\s+(\(?[\d.]+\)?)", teks_bersih, flags=re.IGNORECASE)
          if naik_match:
              raw_naik = naik_match.group(1)
              nilai_naik = int(raw_naik.replace(".", "").replace("(", "").replace(")", ""))
              if "(" in raw_naik:
                  nilai_naik *= -1
          else:
              nilai_naik = None

          turun_match = re.search(r"Rp menurun sebesar\s+\d+%\s+(\(?[\d.]+\)?)", teks_bersih, flags=re.IGNORECASE)
          if turun_match:
              raw_turun = turun_match.group(1)
              nilai_turun = int(raw_turun.replace(".", "").replace("(", "").replace(")", ""))
              if "(" in raw_turun:
                  nilai_turun *= -1
          else:
              nilai_turun = None

      elif perusahaan == "CMNP" :
          match = re.search(r"1\s+Dolar\s+Amerika\s+Serikat\s*\(US\$?\)\s+([\d.,]+)", teks_bersih, re.IGNORECASE)
          if match:
              nilai_naik = match.group(1).replace(".", "").replace(",", ".")
              nilai_turun = nilai_naik


      elif perusahaan in ["SMCB","MDLN"] :
        # Cari baris yang mengandung "Dolar AS - Rupiah"
        if perusahaan == "SMCB" :
          match1 = re.search(r'Dolar AS - Rupiah\s+(\d+\.\d+%)\s+\d+\.\d+%\s+(\(?[\d.,]+\)?)', teks_bersih)
        else :
          match1 = re.search(r'menurun\s+(\d+%)\D+?(\d{1,3}(?:[.]\d{3})+)', teks_bersih)

        if match1:
            persen = match1.group(1)
            nilai_naik = match1.group(2)
            nilai_turun = nilai_naik  # anggap sama dulu
            print("INI BERHASIL")
        else:
            persen = "-"
            nilai_naik = "-"
            nilai_turun = "="
            print("INI GAGAL")


      rupiah_naik = standarisasi_nilai(perusahaan, int(nilai_naik))
      rupiah_turun = standarisasi_nilai(perusahaan, int(nilai_turun))
      rupiah = rupiah_naik
      company.perubahan_laba_rugi_turun = rupiah_turun
      company.normalisasi_laba_rugi_turun = rupiah_turun
      company.normalisasi_laba_rugi_naik = rupiah_naik

    company.satuan_nilai_tukar = persen
    company.perubahan_laba_rugi_naik = rupiah



In [697]:
def cek_fungsi(teks_bersih, perusahaan, kuartal):
    # Cari baris yang mengandung "Dolar AS - Rupiah"
    match = re.search(r'Dolar AS - Rupiah\s+(\d+\.\d+%)\s+\d+\.\d+%\s+(\(?[\d.,]+\)?)', teks_bersih)

    if match:
        # persen = match.group(1)
        nilai_naik = match.group(2)
        nilai_turun = nilai_naik  # anggap sama dulu
    else:
        persen = "-"
        nilai_naik = "-"
        nilai_turun = "="

    # Konversi nilai ke rupiah menggunakan fungsi standarisasi_nilai
    rupiah_naik = standarisasi_nilai(perusahaan, nilai_naik)
    rupiah_turun = standarisasi_nilai(perusahaan, nilai_turun)

    # print("PERSEN " + persen)
    print("NILAI NAIK " + rupiah_naik)
    print("NILAI TURUN " + rupiah_turun)


In [698]:
teks_bersih =""""
ii.  Foreign exchange risk management             Tabel berikut menunjukkan sensitivitas atas  perubahan yang wajar dari nilai tukar Dolar  AS dan Euro terhadap Rupiah, dimana  semua variabel lain konstan, yang timbul dari  aset dan liabilitas dalam Dolar AS dan Euro  terhadap laba sebelum beban pajak untuk  periode  yang  berakhir  pada  tanggal  31 Maret 2024 dan 2023:  The following table demonstrates the sensitivity  to plausible changes in US Dollar and Euro  exchange rates against Rupiah, with all other  variables held constant, arising from the US  Dollar and Euro denominated assets and  liabilities, to the profit before tax for the periods  ended March 31, 2024 and 2023:          Persentase kenaikan (penurunan)/     Efek terhadap laba sebelum pajak/      Increase (decrease) in percentage     Effect on profit before tax        2024     2023     2024     2023            Dolar AS - Rupiah      1.00%     1.00%     1,017      408    US Dollar - Rupiah         -1.00%     -1.00%     (1,017)     (408)      Euro - Rupiah      1.00%     1.00%     (1,021)                          (1,076)  Euro - Rupiah         -1.00%     -1.00%      1,021                            1,076    Aset dan liabilitas moneter yang signifikan  dari Grup dalam mata uang asing pada  tanggal 31 Maret 2024 disajikan pada  Catatan 36.  The Group’s significant monetary assets and  liabilities denominated in foreign currencies as  of March 31, 2024 are presented in Note 36.      Grup hanya melakukan kontrak instrumen  keuangan  derivatif  untuk  melindungi  eksposur yang mendasarinya (“underlying”).  Instrumen keuangan derivatif diukur sebesar  nilai wajarnya.  The Group only enters into derivative financial  instrument  contracts  in  order  to  hedge  underlying  exposures.  Derivative  financial  instruments are recognised at their fair values.      Grup mengikuti Peraturan Bank Indonesia  (“PBI”)  untuk  melakukan  lindung  nilai  terhadap eksposur nilai tukar mata uang  asing selama satu tahun.  The Group follows Bank Indonesia Regulation  (“PBI”) to hedge foreign exchange exposure for  a year.               PT SOLUSI BANGUN INDONESIA TBK  DAN ENTITAS ANAK/AND SUBSIDIARIES    Lampiran 5/77 Schedule    CATATAN ATAS LAPORAN KEUANGAN   KONSOLIDASIAN INTERIM  TANGGAL 31 MARET 2024 DAN 31 DESEMBER 2023  SERTA UNTUK PERIODE TIGA BULAN YANG   BERAKHIR 31 MARET 2024 DAN 2023  (Disajikan dalam jutaan Rupiah,  kecuali dinyatakan lain)  NOTES TO THE INTERIM CONSOLIDATED  FINANCIAL STATEMENTS AS OF MARCH 31, 2024 AND DECEMBER 31, 2023   AND FOR THE THREE-MONTH PERIODS ENDED  MARCH 31, 2024 AND 2023  (Expressed in millions of Rupiah,  unless otherwise stated)       37. INSTRUMEN KEUANGAN, MANAJEMEN RISIKO  KEUANGAN DAN RISIKO MODAL (lanjutan)  37. FINANCIAL INSTRUMENTS, FINANCIAL RISK AND  CAPITAL RISK MANAGEMENT (continued)      b. Tujuan dan kebijakan manajemen risiko  keuangan (lanjutan)  b. Financial risk management objectives and  policies (continued)      iii."""

cek_fungsi(teks_bersih,"SMCB","2024-Q3")

NILAI NAIK Rp1,017,000,000
NILAI TURUN Rp1,017,000,000


In [699]:
def cari_match_perubahan_satuan_nilai_tukar(teks):
    pola_persen = [
        r"(\d+(?:[.,]\d+)?%)",
        r"Dolar A\.S\.\s+\(penguatan\s+(\d+%)\)\s+\(?(-?\d+[\.,]?\d*)\)?",
        r"(\d+\s*basis\s*point)"
    ]

    for pola in pola_persen:
        match = re.search(pola, teks, re.IGNORECASE)
        if match:
            return match.group(1)
    return "-"

### Ekstraksi semua info

In [700]:
def count_elastisity(company_obj: dict):
    print(company_obj)
    def bersihkan_angka(nilai_str):
      if not nilai_str or nilai_str == "-":
          return None
      if isinstance(nilai_str, (int, float)):
          return float(nilai_str)

      # Bersihkan titik ribuan dan karakter non-numeric
      cleaned = (
          str(nilai_str)
          .replace("Rp", "")
          .replace(",", "")
          .replace("%", "")
          .replace(".", "")  # hapus titik ribuan
          .strip()
      )

      # Jika setelah dibersihkan bukan angka valid, tangani error
      try:
          return float(cleaned)
      except ValueError:
          print(f"[!] Tidak bisa konversi ke float: '{nilai_str}' -> '{cleaned}'")
          return None

    persen_perubahan_nilai_tukar = bersihkan_angka(company_obj.get("satuan_nilai_tukar", "-"))
    laba_rugi_naik_after = bersihkan_angka(company_obj.get("normalisasi_laba_rugi_naik", "-"))
    laba_rugi_naik_before = bersihkan_angka(company_obj.get("normalisasi_laba_rugi_tahun_sebelumnya_naik", "-"))
    laba_rugi_turun_after = bersihkan_angka(company_obj.get("normalisasi_laba_rugi_turun", "-"))
    laba_rugi_turun_before = bersihkan_angka(company_obj.get("normalisasi_laba_rugi_tahun_sebelumnya_turun", "-"))

    print (f"Persen perubahan nilai tukar: {persen_perubahan_nilai_tukar}")
    print (f"Laba rugi naik setelah: {laba_rugi_naik_after}")
    print (f"Laba rugi naik sebelum: {laba_rugi_naik_before}")
    print (f"Laba rugi turun setelah: {laba_rugi_turun_after}")
    print (f"Laba rugi turun sebelum: {laba_rugi_turun_before}")

    try:
        perubahan_laba_rugi_naik = laba_rugi_naik_after - laba_rugi_naik_before
        perubahan_laba_rugi_turun = laba_rugi_turun_after - laba_rugi_turun_before

        perubahan_persen_laba_rugi_naik = (perubahan_laba_rugi_naik / abs(laba_rugi_naik_after)) * 100
        perubahan_persen_laba_rugi_turun = (perubahan_laba_rugi_turun / abs(laba_rugi_turun_after)) * 100

        elastisitas_naik = round(persen_perubahan_nilai_tukar / perubahan_persen_laba_rugi_naik, 4)
        elastisitas_turun = round(persen_perubahan_nilai_tukar / perubahan_persen_laba_rugi_turun, 4)

        return elastisitas_naik, elastisitas_turun

    except Exception as e:
        print(f"Error menghitung elastisitas: {e}")
        return "-", "-"

In [701]:
# Reset data supaya fresh sebelum mulai batch baru
all_results_dicts_dinamis.clear()
benchmark_data.clear()

In [675]:
# Reset data supaya fresh sebelum mulai batch baru
all_results_dicts_dinamis.clear()
benchmark_data.clear()

for batch_index, batch_emiten in enumerate(emiten_batches, start=1):
    print(f"\n🚀 Mulai Batch {batch_index}: {batch_emiten}")
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [
            executor.submit(process_batch, batch_emiten, year, quarter)
            for year in years
            for quarter in quarters
        ]

        # 1️⃣ Kumpulkan dulu semua hasil ke 1 list
        all_batch_results = []
        for future in as_completed(futures):
            batch_results = future.result()
            all_batch_results.extend(batch_results)

        # 2️⃣ Urutkan berdasarkan key (emiten-year-quarter)
        all_batch_results.sort(key=lambda x: x[0])

        # 3️⃣ Baru olah satu-satu urut
        for key, data, durasi in all_batch_results:
            all_results_dicts_dinamis[key] = data
            emiten_key, year_key, quarter_key = key.split("-")
            benchmark_data.append({
                "Emiten": emiten_key,
                "Kuartal": f"{year_key}-{quarter_key}",
                "Durasi (detik)": round(durasi, 2)
            })

            tahun_lalu = str(int(year_key) - 1)
            key_tahun_lalu = f"{emiten_key}-{tahun_lalu}-{quarter_key}"
            company_obj = all_results_dicts_dinamis.get(key_tahun_lalu)

            try:
                laba_rugi_naik = company_obj["normalisasi_laba_rugi_naik"]
                laba_rugi_turun = company_obj["normalisasi_laba_rugi_turun"]
                data["normalisasi_laba_rugi_tahun_sebelumnya_naik"] = laba_rugi_naik
                data["normalisasi_laba_rugi_tahun_sebelumnya_turun"] = laba_rugi_turun

            except (TypeError, KeyError):
                laba_rugi_naik = "-"
                laba_rugi_turun = "-"

            with open(output_json_file, "w", encoding="utf-8") as f:
                json.dump(all_results_dicts_dinamis, f, indent=2, ensure_ascii=False)

        for key, data, durasi in all_batch_results:
            try:
                elastisitas_naik, elastisitas_turun = count_elastisity(data)
                data["elastisitas_naik"] = elastisitas_naik
                data["elastisitas_turun"] = elastisitas_turun
                print(f"Elastisitas naik: {elastisitas_naik}")
                print(f"Elastisitas turun: {elastisitas_turun}")

            except (TypeError, KeyError):
                elastisitas_naik = "-"
                elastisitas_turun = "-"

            with open(output_json_file, "w", encoding="utf-8") as f:
                json.dump(all_results_dicts_dinamis, f, indent=2, ensure_ascii=False)



🚀 Mulai Batch 1: ['SMCB']
[⚠️] File tidak ditemukan: /content/drive/MyDrive/IDX_CALK_FX_SB_82/SMCB/SMCB-2021-Q1.pdf
[⚠️] File tidak ditemukan: /content/drive/MyDrive/IDX_CALK_FX_SB_82/SMCB/SMCB-2021-Q2.pdf
[⚠️] File tidak ditemukan: /content/drive/MyDrive/IDX_CALK_FX_SB_82/SMCB/SMCB-2021-Q3.pdf
[⚠️] File tidak ditemukan: /content/drive/MyDrive/IDX_CALK_FX_SB_82/SMCB/SMCB-2021-Q4.pdf
[⚠️] File tidak ditemukan: /content/drive/MyDrive/IDX_CALK_FX_SB_82/SMCB/SMCB-2022-Q1.pdf
[⚠️] File tidak ditemukan: /content/drive/MyDrive/IDX_CALK_FX_SB_82/SMCB/SMCB-2022-Q3.pdf
[⚠️] File tidak ditemukan: /content/drive/MyDrive/IDX_CALK_FX_SB_82/SMCB/SMCB-2022-Q2.pdf
[⚠️] File tidak ditemukan: /content/drive/MyDrive/IDX_CALK_FX_SB_82/SMCB/SMCB-2022-Q4.pdf
[⚠️] File tidak ditemukan: /content/drive/MyDrive/IDX_CALK_FX_SB_82/SMCB/SMCB-2023-Q1.pdf

🔍 Memproses: SMCB - 2023-Q2

🔍 Memproses: SMCB - 2023-Q3
INI KUARTAL:2023-Q2
INI BERHASIL 1
INI BERHASIL
[⚠️] Error pada SMCB-2023-Q2: invalid literal for int() w

Setelah batch selesai diproses, hasil dari setiap `future` yang disubmit akan diambil. Untuk setiap hasil (`key`, `data`, dan `durasi)`, data tersebut disimpan ke dalam `all_results_dicts_dinamis`. Durasi pemrosesan per kuartal juga dicatat dalam benchmark_data.

Setelah pemrosesan satu kuartal selesai, hasilnya disimpan dalam file JSON (`output_json_file`). Ini memastikan bahwa data yang sudah diproses tidak hilang jika program terhenti di tengah jalan.

#### Output & Evaluasi Kinerja

Data yang terkumpul dalam `all_results_dicts_dinamis` diubah menjadi sebuah DataFrame menggunakan pandas. Data ini kemudian disortir berdasarkan kolom Emiten dan Kuartal. Begitu juga dengan data benchmark (`benchmark_data`), yang Setelah data disortir, hasilnya disimpan dalam file Excel untuk data utama (`output_file)` dan file benchmark (`benchmark_file`) juga diubah menjadi DataFrame dan disortir berdasarkan Emiten dan Kuartal.

In [676]:
import re

df = pd.DataFrame(list(all_results_dicts_dinamis.values()))
df.to_excel(output_file, index=False)
print(f"\n✅ File hasil utama: {output_file}")


✅ File hasil utama: hasil_ekstraksi.xlsx


Waktu total yang dibutuhkan untuk menjalankan program dihitung dan dicetak setelah semua batch selesai diproses. Jika kode dijalankan di Google Colab, file hasil (`output_file` dan `benchmark_file`) akan diunduh secara otomatis ke mesin lokal pengguna.

In [677]:
# Urutkan benchmark berdasarkan 'Emiten' dan 'Kuartal'
benchmark_df = pd.DataFrame(benchmark_data)
# benchmark_df.sort_values(by=["Emiten", "Kuartal"], inplace=True)

print(f"\n✅ File hasil utama: {output_file}")
benchmark_df.to_excel(benchmark_file, index=False)


print(f"⏱️ File benchmark waktu dibuat: {benchmark_file}")

# Waktu total program
end_program_time = time.time()
print(f"\n🕒 Total waktu runtime program: {round(end_program_time - start_program_time, 2)} detik")

# Auto-download jika di Colab
try:
    from google.colab import files
    files.download(output_file)
    files.download(benchmark_file)
except:
    pass


✅ File hasil utama: hasil_ekstraksi.xlsx
⏱️ File benchmark waktu dibuat: benchmark_waktu.xlsx

🕒 Total waktu runtime program: 9.91 detik


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Process Documentation

## 1. CAPTURE – Menentukan Kebutuhan dan Target Informasi


**Tujuan:**  
Memastikan informasi yang dibutuhkan untuk analisis dan pemantauan sektor korporasi.

**Proses:**
- Identifikasi kebutuhan informasi kebijakan yang ingin diekstrak dari laporan keuangan, misalnya: risiko nilai tukar, suku bunga.
- Pengumpulan dokumen laporan keuangan dari Web IDX Indonesia.
- Penetapan marker dan keyword pencarian, seperti: `nilai tukar`, `mata uang asing`, dan sebagainya.
- Pembacaan PDF halaman demi halaman.
- Pencocokan marker berdasarkan konfigurasi per perusahaan untuk menemukan paragraf relevan.
- Ekstraksi paragraf yang mengandung informasi sesuai marker.

## 2. COMPILE – Ekstraksi dan Strukturisasi Data Otomatis

**Tujuan:**  
Mengubah dokumen unstructured menjadi data terstruktur dan siap dianalisis.

**Proses:**
- Praproses teks: normalisasi whitespace, penghapusan header/footer, dan filter karakter asing.
- Identifikasi entitas penting seperti nama mata uang, nilai eksposur, dan instrumen lindung nilai.
- Strukturisasi hasil ekstraksi ke dalam format dictionary atau JSON.

## 3. ANALYSIS – Pemanfaatan Data untuk Insight dan Kebijakan

**Tujuan:**  
Menyajikan insight relevan, cepat, dan konsisten untuk mendukung keputusan makroprudensial.

**Proses:**
- Agregasi data antar perusahaan dan kuartal untuk mengidentifikasi tren sektoral.
- Pemetaan eksposur nilai tukar berdasarkan sektor, industri, atau perusahaan besar.
- Visualisasi data dan pengembangan dashboard monitoring.
- Penyusunan narasi untuk CHAF atau Kajian Stabilitas Keuangan (KSK).
- Penyampaian rekomendasi kebijakan berbasis evidence kepada stakeholder internal atau eksternal.